In [6]:

import scipeds

scipeds.download_db()

from scipeds.data.completions import CompletionsQueryEngine
from scipeds.data.queries import (
    FieldTaxonomy,
    QueryFilters, 
)

engine = CompletionsQueryEngine()



Database already downloaded to /Users/kevinphan981/Library/Caches/.scipeds/scipeds_0_0_8.duckdb. To re-download and overwrite the existing file, re-run with `overwrite` set to `True`(from within Python) or add --overwrite to your CLI command.


In [7]:
from scipeds.constants import COMPLETIONS_TABLE

completions = engine.get_df_from_query(f"""
    SELECT year, unitid, cipcode, awlevel, majornum, cip2020, race_ethnicity, gender, n_awards  
    FROM {COMPLETIONS_TABLE}
    WHERE YEAR BETWEEN 2010 AND 2019
    AND regexp_matches(CAST(CIPCODE AS VARCHAR), '^12\\.04.*$')
    ;
""")
print(completions.head())
print(completions.shape)

   year  unitid  cipcode                                            awlevel  \
0  2010  100760  12.0401  Award of at least 1 but less than 2 academic y...   
1  2010  101240  12.0401                 Award of less than 1 academic year   
2  2010  101240  12.0401                 Award of less than 1 academic year   
3  2010  101240  12.0401                 Award of less than 1 academic year   
4  2010  101240  12.0401                 Award of less than 1 academic year   

   majornum  cip2020             race_ethnicity gender  n_awards  
0         1  12.0401  Black or African American  women         3  
1         1  12.0401                    Unknown  women         1  
2         1  12.0401  Black or African American    men         1  
3         1  12.0401  Black or African American  women         2  
4         1  12.0401         Hispanic or Latino  women         1  
(146367, 9)


In [8]:
# to see the tables available

query = "SHOW TABLES"

engine.get_df_from_query(query) # this package doesn't offer that much, but it does at least offer the completions data

,name
0,cip_info
1,ipeds_completions_a
2,ipeds_directory_info


In [10]:
institution_fip = engine.get_institutions_table() # reads in all institutions and their features (we need state)
print(institution_fip.shape)

(9469, 102)


In [ ]:
# creating index of cosmetology institutions
import pandas as pd
# cosmetology_schools = pd.merge(institution_fip, completions, how = "left", on = "unitid")
# wrong idea here, we need the unique schools from 'completions'

cosmetology_schools = completions['unitid'].unique()

print(cosmetology_schools.shape) # we cut down the amount by nearly a fourth


(2474,)


## Switching Gears to Pypeds

This is another library to help read in the data from IPEDS en masse.

In [2]:
import pypeds
from pypeds import ipeds

timeframe = range(2010, 2019, 1)
enrollment = ipeds.EFC(years = timeframe)
enrollment.extract()

enrollment_df = enrollment.load()

In [4]:
enrollment_df.head()
print(enrollment_df.shape)

(526136, 9)
